# Storing Human Ageing Genomic data using Hyperledger Fabric

### Client

In [1]:
#import hyperledger fabric library
from hfc.fabric import Client

# Define client object to access fabric network functions
cli = Client(net_profile="test/fixtures/network.json")

print(cli.organizations)  # orgs in the network
print(cli.peers)  # peers in the network
print(cli.orderers)  # orderers in the network
print(cli.CAs)  # ca nodes in the network

# Define admins for the organisations and ordering service
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin') # User instance with the Org1 admin's certs
org2_admin = cli.get_user(org_name='org2.example.com', name='Admin') # User instance with the Org2 admin's certs
orderer_admin = cli.get_user(org_name='orderer.example.com', name='Admin') # User instance with the orderer's certs

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


{'orderer.example.com': <hfc.fabric.organization.Organization object at 0x7fec5043b8b0>, 'org1.example.com': <hfc.fabric.organization.Organization object at 0x7fec31127b20>, 'org2.example.com': <hfc.fabric.organization.Organization object at 0x7fec31127df0>}
{'peer0.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fec3112b640>, 'peer1.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fec3112bd00>, 'peer0.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fec3112bf10>, 'peer1.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fec3112b9d0>}
{'orderer.example.com': <hfc.fabric.orderer.Orderer object at 0x7fec50480ac0>}
{'ca-org1': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fec3112b1c0>, 'ca-org2': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fec3112b1f0>}


### Create a Healthcare channel and let the users(peers) join the channel

In [3]:
response = await cli.channel_create(
                    orderer='orderer.example.com',
                    channel_name='healthcarechannel',
                    requestor=org1_admin,
                    config_yaml='test/fixtures/e2e_cli/',
                    channel_profile='TwoOrgsChannel'
                    )

# response = true is returned if the channel is created successfully
print(response==True)

responses = await cli.channel_join(
                       requestor=org1_admin,
                       channel_name='healthcarechannel',
                       peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

responses = await cli.channel_join(
                       requestor=org2_admin,
                       channel_name='healthcarechannel',
                       peers=['peer0.org2.example.com',
                      'peer1.org2.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

FABRIC_CFG_PATH set to /home/ghanendra/fabric-sdk-py/test/fixtures/e2e_cli/
Configtx file successfully created in current             directory
{'tx_id': 'e50084eff936ec6e035385ee44479521cdf7c4e12ebd4c091cf9e08126867418', 'nonce': b'\x17C\x16\x7f7t(/\x07\xb2t\x07\xfd\x1a\xd7R\xde\x10\x04\xba\xc9E\xd3\xe8', 'signatures': [b'\n\xd3\x06\n\xb6\x06\n\x07Org1MSP\x12\xaa\x06-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMNU2FuIEZy\nYW5jaXNjbzEZMBcGA1UEChMQb3JnMS5leGFtcGxlLmNvbTEcMBoGA1UEAxMTY2Eu\nb3JnMS5leGFtcGxlLmNvbTAeFw0xODEwMTkwMzQ4MDBaFw0yODEwMTYwMzQ4MDBa\nMGwxCzAJBgNVBAYTAlVTMRMwEQYDVQQIEwpDYWxpZm9ybmlhMRYwFAYDVQQHEw1T\nYW4gRnJhbmNpc2NvMQ8wDQYDVQQLEwZjbGllbnQxHzAdBgNVBAMMFkFkbWluQG9y\nZzEuZXhhbXBsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAR2zFPDBYIy\njZSXaB15ILW9MRUYkSiksD6Io+VBbsJex2S7Do7lJhYfpyg5Z1LEHLCQgHH/VJ1F\nMZmNHgIIHqWPo00wSzAOBgNVHQ8BAf8EBAMCB4AwDAYDVR0TAQH/BAIwADArBgNV\nHSMEJDA

True


get genesis block successfully, block=data_hash: "D\021\0349\266\225\342\026Li\221lh\317\221i\022\322\243\373\252\236E\214V\353\202\375o\026eO"



True
True


### Install Chaincode to peers of organisation 1

In [4]:
# This installs the example chaincode on the peers
# Make the client know there is a channel in the network
cli.new_channel('healthcarechannel')

#The GOPTAH settings is required for only running the example chaincode in the SDK
import os
gopath_bak = os.environ.get('GOPATH', '')
gopath = os.path.normpath(os.path.join(
                      os.path.dirname(os.path.realpath('__file__')),
                      'test/fixtures/chaincode/'
                     ))

os.environ['GOPATH'] = os.path.abspath(gopath)

# The response should be true (status 200)
response = await cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/genomic_cc',
               cc_name='genomic_cc',
               cc_version='v1.0'
               )
print(response)

New channel with name = healthcarechannel


[response {
  status: 200
  payload: "OK"
}
, response {
  status: 200
  payload: "OK"
}
]


### Install Chaincode to peers of organisation 2

In [5]:
# The response should be true (status 200)
response = await cli.chaincode_install(
               requestor=org2_admin,
               peers=['peer0.org2.example.com',
                      'peer1.org2.example.com'],
               cc_path='github.com/genomic_cc',
               cc_name='genomic_cc',
               cc_version='v1.0'
               )
print(response)

[response {
  status: 200
  payload: "OK"
}
, response {
  status: 200
  payload: "OK"
}
]


### Query Installed Chaincodes

In [9]:
#Query Installed Chaincodes

response = await cli.query_installed_chaincodes(
    requestor=org1_admin,
    peers=['peer0.org1.example.com','peer1.org1.example.com'],
    )
print(response)

response = await cli.query_installed_chaincodes(
    requestor=org2_admin,
    peers=['peer0.org2.example.com','peer1.org2.example.com'],
    )
print(response)

cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc


[chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  id: "\211e\345\037\331\007Y\230W\001^R\341\\\322c.E4\221\r\213\361ts/\227)N\364\231\371"
}
, chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  id: "\211e\345\037\331\007Y\230W\001^R\341\\\322c.E4\221\r\213\361ts/\227)N\364\231\371"
}
]
[chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  id: "\211e\345\037\331\007Y\230W\001^R\341\\\322c.E4\221\r\213\361ts/\227)N\364\231\371"
}
, chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  id: "\211e\345\037\331\007Y\230W\001^R\341\\\322c.E4\221\r\213\361ts/\227)N\364\231\371"
}
]


### Instantiate Chaincode in Channel

In [10]:
#Instantiate Chaincode in Channel, the response should be true if succeed

# This policy specifies the endorsement policy which is required while instantiating the chaincode
policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0},
        ]
    }
}


cc_config = [
    {
         "name": "collectionGenes",
         "policy": policy,
         "requiredPeerCount": 0,
         "maxPeerCount": 3,
         "blockToLive":1000000,
         "memberOnlyRead": True
    },
  
    {
         "name": "collectionGenesPrivateDetails",
         "policy": policy,
         "requiredPeerCount": 0,
         "maxPeerCount": 3,
         "blockToLive":3,
         "memberOnlyRead": True
    }

]

#args = ['1', 'Ron', 'Finnish', 'Mec1','99','23','Mec2','15']
# gene1 = '{"id":12,"name":"Dennis","population":"Americana","gene":"APO1","size":320,"age":27,"varient":"APO3","price":91}'
# tmap = {'gene':gene1.encode()}


response = await cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=None,
               cc_name='genomic_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=cc_config, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               )
print(response)

{'name': 'genomic_cc', 'version': 'v1.0', 'escc': 'escc', 'vscc': 'vscc', 'policy': {'version': 0, 'rule': {'n_out_of': {'n': 1, 'rules': [{'signed_by': 0}]}}, 'identities': [{'principal_classification': 'ROLE', 'principal': {'msp_identifier': 'Org1MSP', 'role': 'MEMBER'}}]}, 'data': {'hash': b'\x9d\xc4\xd4,\x08\x93\xcc\xa6>\xea\xa9\xf6\xe3\x0f9\xd5\xcf.\x87\x9b\xd2\xb3\xb8w\xfa\xeail\xfdK\xc9 ', 'metadatahash': b'xI\x91\x83\xe5\x9b\xb3\n\x85\x99\xe0[\xad\xbd\xa2\x084e\x1e\xfc\xc1R>\x96\xf1\x80/\x80\x1a\x0c\x13\xc2'}, 'id': b'\x89e\xe5\x1f\xd9\x07Y\x98W\x01^R\xe1\\\xd2c.E4\x91\r\x8b\xf1ts/\x97)N\xf4\x99\xf9', 'instantiation_policy': {'version': 0, 'rule': {'n_out_of': {'n': 1, 'rules': [{'signed_by': 0}, {'signed_by': 1}]}}, 'identities': [{'principal_classification': 'ROLE', 'principal': {'msp_identifier': 'Org1MSP', 'role': 'ADMIN'}}, {'principal_classification': 'ROLE', 'principal': {'msp_identifier': 'Org2MSP', 'role': 'ADMIN'}}]}}


### Query Instantiated Chaincodes

In [12]:
response = await cli.query_instantiated_chaincodes(
    requestor=org2_admin,
    channel_name='healthcarechannel',
    peers=['peer0.org1.example.com','peer1.org1.example.com','peer0.org2.example.com','peer1.org2.example.com'],
    )
print(response)

cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc
cc name genomic_cc, version v1.0, path github.com/genomic_cc


[chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  input: "<nil>"
  escc: "escc"
  vscc: "vscc"
}
, chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  input: "<nil>"
  escc: "escc"
  vscc: "vscc"
}
, chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  input: "<nil>"
  escc: "escc"
  vscc: "vscc"
}
, chaincodes {
  name: "genomic_cc"
  version: "v1.0"
  path: "github.com/genomic_cc"
  input: "<nil>"
  escc: "escc"
  vscc: "vscc"
}
]


### Initialize a gene

In [19]:
# Initialize a gene

gene1 = '{"id":12,"name":"Dennis","population":"American","gene":"APO1","size":320,"age":27,"varient":"APO3","price":91}'
tmap = {'gene':gene1.encode()}

response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=None,
               cc_name='genomic_cc',
               fcn = "initGene",
               transient_map=tmap, # optional, for private data
               wait_for_event=True,
                )
print(response)


gene1 = '{"id":17,"name":"John","population":"Dutch","gene":"AFZE","size":150,"age":33,"varient":"AFEE","price":33}'
tmap = {'gene':gene1.encode()}

response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=None,
               cc_name='genomic_cc',
               fcn = "initGene",
               transient_map=tmap, # optional, for private data
               wait_for_event=True,
               )
print(response)


This gene already exists: John


### Initialize couple of genomic information to database

In [56]:
gene_A = '{"id":21,"name":"Donghoon","population":"Korean","gene":"MEC1","size":120,"age":38,"varient":"MEC3","price":89}'
gene_B = '{"id":22,"name":"Shubham","population":"Indian","gene":"TEL1","size":140,"age":22,"varient":"TEL5","price":55}'
gene_C = '{"id":23,"name":"Tanmay","population":"American","gene":"TELO","size":160,"age":23,"varient":"ZELO","price":39}'
gene_D = '{"id":24,"name":"Munawar","population":"German","gene":"CDC13","size":115,"age":29,"varient":"CDK1","price":41}'
gene_E = '{"id":25,"name":"Ghanendra","population":"French","gene":"RIF1","size":125,"age":26,"varient":"RIF2","price":57}'

tmapA = {'gene':gene_A.encode()}
tmapB = {'gene':gene_B.encode()}
tmapC = {'gene':gene_C.encode()}
tmapD = {'gene':gene_D.encode()}
tmapE = {'gene':gene_E.encode()}

dict_tmaps = {'tmapA':tmapA,'tmapB':tmapB,'tmapC':tmapC,'tmapD':tmapD,'tmapE':tmapE}

for tmap_index,tmap_value in dict_tmaps.items():

    response = await cli.chaincode_invoke(
                   requestor=org1_admin,
                   channel_name='healthcarechannel',
                   peers=['peer0.org1.example.com'],
                   args=None,
                   cc_name='genomic_cc',
                   fcn = "initGene",
                   transient_map=tmap_value, # optional, for private data
                   wait_for_event=True,
                    )
    print(response)
    print('Gene added to CouchDB:',tmap_index)



Gene added to CouchDB: tmapA

Gene added to CouchDB: tmapB

Gene added to CouchDB: tmapC

Gene added to CouchDB: tmapD

Gene added to CouchDB: tmapE


### Read a gene

In [63]:
# Read a gene
args = ["Dennis"]

# The response must be genomic information
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='genomic_cc',
               fcn = "readGene",
               transient_map=None, # optional, for private data
               wait_for_event=True,
               )
print(response)

{"docType":"gene","id":12,"Name":"Dennis","population":"American","gene":"ATTCGGATAACGCG","size":320}


In [66]:
args = [["Donghoon"],["Tanmay"],["Shubham"],["Munawar"],["Ghanendra"]]

# Read a couple of genes
for args_i in args:

    # The response must be genomic information by a person's name
    response = await cli.chaincode_invoke(
                   requestor=org1_admin,
                   channel_name='healthcarechannel',
                   peers=['peer0.org1.example.com'],
                   args=args_i,
                   cc_name='genomic_cc',
                   fcn = "readGene",
                   transient_map=None, # optional, for private data
                   wait_for_event=True,
                   )
    print('%s : '%args_i,response)

['Donghoon'] :  {"Name":"Donghoon","docType":"gene","gene":"MEC1","id":21,"population":"Korean","size":120}
['Tanmay'] :  {"Name":"Tanmay","docType":"gene","gene":"TELO","id":23,"population":"American","size":160}
['Shubham'] :  {"Name":"Shubham","docType":"gene","gene":"TEL1","id":22,"population":"Indian","size":140}
['Munawar'] :  {"Name":"Munawar","docType":"gene","gene":"CDC13","id":24,"population":"German","size":115}
['Ghanendra'] :  {"Name":"Ghanendra","docType":"gene","gene":"RIF1","id":25,"population":"French","size":125}


### Read private details of a gene

In [15]:
# Read private details of a gene
args = ["Dennis"]

# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='genomic_cc',
               fcn = "readGenePrivateDetails",
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger
               )
print(response)

{"age":27,"docType":"genePrivateDetails","name":"Dennis","price":91,"varient":"APO3"}


### Delete a gene

In [16]:
# Delete a gene
args = ["Dennis"]

tmap_gd = {"gene_delete":b'{"name":"Dennis"}'}

# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=None,
               cc_name='genomic_cc',
               fcn = "delete",
               transient_map=tmap_gd, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger
               )
print(response)

### Transfer Gene data

In [20]:
# Transfer Gene data
# Gene Therapy
tmap_gt = {"gene_name":b'{"gene":"ATTCGGATAACGCG","name":"Dennis"}'}

# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=None,
               cc_name='genomic_cc',
               fcn = "transferGene",
               transient_map=tmap_gt, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger
               )
print(response)

### Get Genes data by Name

In [21]:
# Get Genes data by Name

#name":"Dennis","age":27,"varient":"APO3","price":91}
args = ["A","Z"]

# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='genomic_cc',
               fcn = "getGenesByRange",
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger
               )
print(response)

[{"Key":"Dennis", "Record":{"Name":"Dennis","docType":"gene","gene":"ATTCGGATAACGCG","id":12,"population":"American","size":320}},{"Key":"John", "Record":{"Name":"John","docType":"gene","gene":"AFZE","id":17,"population":"Dutch","size":150}}]


### Query LongetivityMap data By Gene

In [25]:
# Query Entire data By Gene

#Query all the data stores as Key and Record
#args = ['{"selector":{}}']

args = ['{"selector":{"gene":"ATTCGGATAACGCG"}}']

# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='healthcarechannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='genomic_cc',
               fcn = "queryLongetivityMapByGene",
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger
               )
print(response)

[{"Key":"Dennis", "Record":{"Name":"Dennis","docType":"gene","gene":"ATTCGGATAACGCG","id":12,"population":"American","size":320}}]
